<h1>Импорты</h1>

In [87]:
import pandas as pd
import numpy as np
import pyodbc
import random
import math
from sklearn.preprocessing import OrdinalEncoder
from datetime import datetime

In [88]:
import warnings
warnings.filterwarnings('ignore')

In [89]:
# Connect to old database
db_conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=BAMBI;'
                          'Database=Pro_t;'
                          'Trusted_Connection=yes;')

In [90]:
# Connect to archive
archive_conn = pyodbc.connect('Driver={SQL Server};'
                              'Server=progress;'
                              'Database=ArchDocT;'
                              'Trusted_Connection=yes;')

In [91]:
# Connect to new database
try:
    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=FAST;'
#                           'Server=BAMBI;'
                          'Database=DocumentsKM;'
                          'Trusted_Connection=yes;')
    cursor = conn.cursor()
except Exception as err:
    cursor = None
    print("Connection error:", err)
    
# Check if the connection was valid
if cursor != None:
    print("Connection successful")

Connection successful


In [92]:
def to_pascal_case(snake_str):
    components = snake_str.split('_')
    return components[0].title() + ''.join(x.title() for x in components[1:])

In [93]:
def execute_values(df, table_snake_case, has_dentity=True):
    df.columns = map(to_pascal_case, df.columns)
    df = df.replace({False: 0, True: 1})
    table = to_pascal_case(table_snake_case)
    if has_dentity:
        cursor.execute(f"SET IDENTITY_INSERT {table} ON")
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    for entry in tuples:
        query  = "insert into %s(%s) values (" % (table, cols)
        num = 1
        entry_list = []
        for e in entry:
            if num == len(entry):
                query += "?"
            else:
                query += "?, "
            num += 1
            if (not isinstance(e, str)) and (not isinstance(e, bool)) and (not (e is None)) and (not (type(e) is pd.Timestamp)) and math.isnan(e):
                entry_list.append(None)
            else:
                entry_list.append(e)
        query += ")"
        cursor.execute(query, *entry_list)
    if has_dentity:
        cursor.execute(f"SET IDENTITY_INSERT {table} OFF")
    conn.commit()
    cursor.execute(f"ALTER SCHEMA dbo TRANSFER [GIPROMEZ\ВладАИ].[{table}]")
    conn.commit()

To delete:
    1) Select Tables
    2) Press F7
    3) Select tables
    4) Delete them till it's done

<h1>Исполнители старые id в новые</h1>

In [94]:
# df = pd.read_sql_query('SELECT * FROM DocumentsKM.dbo.Old_New', conn)

In [95]:
# df.to_csv('old_new.csv', index=False)

In [96]:
new_old_employees = pd.read_csv("old_new.csv")

In [97]:
def to_new_employee_ids(df, column_name):
    return df[column_name].map(lambda x: list(new_old_employees.loc[new_old_employees["OldId"] == x, "NewId"])[0] if list(new_old_employees.loc[new_old_employees["OldId"] == x, "NewId"]) else None)

<h1>Подузлы марок</h1>

In [98]:
# df = pd.read_sql_query('SELECT * FROM DocumentsKM.dbo.Marks', conn)

In [99]:
# df = df.loc[:, ["Id", "ArchSubnodeId", "ArchMarkId"]]

In [100]:
# df.to_csv('mark_archive.csv', index=False)

In [101]:
mark_archive = pd.read_csv("mark_archive.csv")

<h1>Архив</h1>

In [102]:
archive_nodes = pd.read_sql_query('SELECT * FROM ArchDocT.dbo.Узлы', archive_conn)

In [103]:
archive_subnodes = pd.read_sql_query('SELECT * FROM ArchDocT.dbo.Подузлы', archive_conn)

In [104]:
archive_marks = pd.read_sql_query('SELECT * FROM ArchDocT.dbo.МаркиПроекта', archive_conn)

In [105]:
archive_nodes = archive_nodes.loc[:, ["Узел", "ГИП"]]

In [106]:
archive_subnodes = archive_subnodes.loc[:, ["Подузел", "Узел"]]

In [107]:
archive_marks = archive_marks.loc[:, ["Марка", "Подузел", "НазвМарки"]]

<h1>Должности</h1>

In [108]:
df = pd.read_json('post.json')

In [109]:
df["name"] = df["name"].str.strip()

In [110]:
cursor.execute(open("sql_server/25.sql", "r").read())
conn.commit()

In [111]:
execute_values(df, "positions", False)

<h1>Отделы</h1>

In [112]:
df = pd.read_json('department.json')

In [113]:
df["name"] = df["name"].str.strip()
df["reduction"] = df["reduction"].str.strip()

In [114]:
df = df.rename(columns={"id" : "code",
                        "reduction": "short_name",
                        "enable": "is_active"
                       })

In [115]:
df.insert(0, 'id', range(1, len(df) + 1))

In [116]:
df.loc[:, ["id", "code"]].to_csv("departments.csv", index=False)

In [117]:
cursor.execute(open("sql_server/30.sql", "r").read())
conn.commit()

In [118]:
execute_values(df, "departments")

<h1>Исполнители</h1>

In [119]:
df = pd.read_json('staff.json')

In [120]:
df_dep = df['department'].apply(pd.Series)

In [121]:
df_dep = df_dep.drop(["name", "reduction", "enable"], axis=1)

In [122]:
df_dep = df_dep.rename(columns={"id": "code"})

In [123]:
dep = pd.read_csv("departments.csv")

In [124]:
for index, row in df_dep.iterrows():
    try:
        df_dep.loc[index, "department_id"] = dep.loc[dep["code"] == row["code"], "id"].values[0]
    except:
        df_dep.loc[index, "department_id"] = None

In [125]:
df_dep = df_dep.drop(["code"], axis=1)

In [126]:
df["department_id"] = df_dep

In [127]:
df_post = df['post'].apply(pd.Series)

In [128]:
df_post = df_post.drop(["name"], axis=1)

In [129]:
df_post = df_post.rename(columns={"id": "position_id"})

In [130]:
df["position_id"] = df_post

In [131]:
df["is_active"] = df["dismissedDate"].isna()

In [132]:
df = df.drop(["tableNumber", "userName", "createdDate", "dismissedDate", "vacation", "post", "department"], axis=1)

In [133]:
df = df.dropna()

In [134]:
df["name"] = df["fullName"].str.partition().loc[:, 0]

In [135]:
df["id"].to_csv("employee_ids.csv", index=False)

In [136]:
cursor.execute(open("sql_server/26.sql", "r").read())
conn.commit()

In [137]:
execute_values(df, "employees", False)

<h1>Пользователи</h1>

In [138]:
df = pd.DataFrame({'id': [1, 2], 'login': ["1", "2"], 'password': ["1", "2"], 'employee_id': [5, 6]})

In [139]:
cursor.execute(open("sql_server/xx_users.sql", "r").read())
conn.commit()

In [140]:
execute_values(df, "users")

<h1>Значения по умолчанию</h1>

In [141]:
df = pd.DataFrame({'user_id': [1, 2], 'department_id': [74, 74], 'creator_id': [5, 6], 'inspector_id': [None, None], 'norm_contr_id': [None, None]})

In [142]:
cursor.execute(open("sql_server/xx_default.sql", "r").read())
conn.commit()

In [143]:
execute_values(df, "default_values", False)

<h1>Условия эксплуатации</h1>

In [144]:
df1 = pd.DataFrame({'id': [1, 2, 3, 4], 'name': ["неагрессивная", "слабоагрессивная", "среднеагрессивная", "сильноагрессивная"]})

In [145]:
df2 = pd.DataFrame({'id': [1, 2, 3], 'name': ["внутри помещения", "снаружи помещения", "в жидкостях"]})

In [146]:
df3 = pd.DataFrame({'id': [1, 2, 3, 4, 5], 'name': ["нет газов (вода)", "газы группы А (кислоты)", "газы группы B, C и D (щелочи)", "бензин", "масло"]})

In [147]:
df4 = pd.DataFrame({'id': [1, 2, 3, 4], 'name': ["сталь без покрытия", "оцинкованная сталь", "сталь горячего цинкования", "сталь с газотермическим напылением цинка"]})

In [148]:
df5 = pd.DataFrame({'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 'name': ["ПФ", "АУ", "ЭЭ", "МА", "МБ", "НЦ", "ФФ", "ПА", "ПЭ", "ПБ", "ПВ", "СВ", "ПУ", "ЭП", "ПС", "КО", "АЦ"]})

In [149]:
df6 = pd.DataFrame({'id': [1, 2, 3, 4], 'name': ["отсутствуют", "сдвигоустойчивые соединения", "фланцевые соединения", "сдвигоустойчивые и фланцевые соединения"]})

In [150]:
df5.loc[:, ["id", "name"]].to_csv("paintwork.csv", index=False)

In [151]:
cursor.execute(open("sql_server/y_1.sql", "r").read())
cursor.execute(open("sql_server/y_2.sql", "r").read())
cursor.execute(open("sql_server/y_3.sql", "r").read())
cursor.execute(open("sql_server/y_4.sql", "r").read())
cursor.execute(open("sql_server/y_5.sql", "r").read())
cursor.execute(open("sql_server/y_6.sql", "r").read())
conn.commit()

In [152]:
execute_values(df1, "env_aggressiveness")

In [153]:
execute_values(df2, "operating_areas")

In [154]:
execute_values(df3, "gas_groups")

In [155]:
execute_values(df4, "construction_materials")

In [156]:
execute_values(df5, "paintwork_types")

In [157]:
execute_values(df6, "high_tensile_bolts_types")

<h1>Стойкость лакокрасочных покрытий</h1>

In [158]:
df = pd.DataFrame({'id': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'name': ["П", "А", "Х", "В", "Б", "М", "ХЩ", "ХК", "Т"]})

In [159]:
df.loc[:, ["id", "name"]].to_csv("paintwork_fastness.csv", index=False)

In [160]:
cursor.execute(open("sql_server/y_7.sql", "r").read())
conn.commit()

In [161]:
execute_values(df, "paintwork_fastness")

<h1>Контроль сварки</h1>

In [162]:
df = pd.DataFrame({'Id': [1, 2, 3, 4], 'Name': ["не требуется", "опрыскиванием керосином", "ультразвуком", "гамма-излучением"]})

In [163]:
cursor.execute(open("sql_server/yy_1.sql", "r").read())
conn.commit()

In [164]:
execute_values(df, "WeldingControl")

<h1>Типы документов</h1>

In [165]:
df = pd.DataFrame({'id': [1, 2, 3, 4, 5], 'code': ["л.", "СМ", "ВМП", "СМС", "РР"], 'name': ["Листы основного комплекта", "Спецификация металлопроката", "Ведомость металлоконструкций по видам профилей", "Сводная спецификация металлопроката", "Расчет металлоконструкций"]})

In [166]:
cursor.execute(open("sql_server/x.sql", "r").read())
conn.commit()

In [167]:
execute_values(df, "doc_types")

<h1>Грунтовка</h1>

In [168]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Грунт_аз', db_conn)

In [169]:
df = df.replace("", np.nan)

In [170]:
df["Приоритет"] = df["Приоритет"].map(lambda x: 0 if pd.isna(x) else x)

In [171]:
df = df.rename(columns={"поз": "id",
                        "Группа_гр": "group_num",
                        "Наим_гр": "name",
                        "Можно_гр": "can_be_primed",
                        "Приоритет": "priority"
                       })

In [172]:
cursor.execute(open("sql_server/4.sql", "r").read())
conn.commit()

In [173]:
execute_values(df, "primer")

<h1>Степени очистки антикоррозионной защиты</h1>

In [174]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Степени_оч_аз', db_conn)

In [175]:
df["Наим_оч"] = df["Наим_оч"].str.strip()

In [176]:
df = df.rename(columns={"Степень_оч": "id",
                        "Наим_оч": "name"
                       })

In [177]:
cursor.execute(open("sql_server/44.sql", "r").read())
conn.commit()

In [178]:
execute_values(df, "corr_prot_cleaning_degrees")

<h1>Варианты антикоррозионной защиты </h1>

In [179]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Варианты_аз', db_conn)

In [180]:
df["Тип_лп"] = df["Тип_лп"].str.strip()
df["Стойкость_лп"] = df["Стойкость_лп"].str.strip()

In [181]:
df = df.replace("", np.nan)

In [182]:
df = df.drop_duplicates()

In [183]:
df = df.drop_duplicates(subset=["Зона_экс", "Группа_газов", "Агресс_ср", "Матер_к", "Тип_лп", "Группа_лп", "Стойкость_лп"])

In [184]:
paintwork = pd.read_csv("paintwork.csv")

In [185]:
for index, row in df.iterrows():
    try:
        df.loc[index, "paintwork_type_id"] = paintwork.loc[paintwork["name"] == row["Тип_лп"], "id"].values[0]
    except:
        df.loc[index, "paintwork_type_id"] = None

In [186]:
paintwork_fastness = pd.read_csv("paintwork_fastness.csv")

In [187]:
for index, row in df.iterrows():
    try:
        df.loc[index, "paintwork_fastness_id"] = paintwork_fastness.loc[paintwork_fastness["name"] == row["Стойкость_лп"], "id"].values[0]
    except:
        df.loc[index, "paintwork_fastness_id"] = None

In [188]:
df = df.drop(["Тип_лп", "Стойкость_лп"], axis=1)

In [189]:
df["Группа_газов"] += 1
df["Агресс_ср"] += 1
df["Матер_к"] += 1

In [190]:
df = df.rename(columns={"Вариант": "id",
                        "Зона_экс": "operating_area_id",
                        "Группа_газов": "gas_group_id",
                        "Агресс_ср": "env_aggressiveness_id",
                        "Матер_к": "construction_material_id",
                        "Группа_лп": "paintwork_group",
                        "Кол_слоев_лп": "paintwork_num_of_layers",
                        "Толщ_лпгр": "paintwork_primer_thickness",
                        "Кол_слоев_гр": "primer_num_of_layers",
                        "Степень_оч": "cleaning_degree_id",
                        "Статус_аз": "status"
                       })

In [191]:
df = df.where(pd.notnull(df), None)

In [192]:
cursor.execute(open("sql_server/1.sql", "r").read())
conn.commit()

In [193]:
execute_values(df, "corr_prot_variants")

<h1>Способы антикоррозионной защиты</h1>

In [194]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Способы_аз', db_conn)

In [195]:
df["Наим_сп"] = df["Наим_сп"].str.strip()

In [196]:
df["Агресс_ср"] += 1
df["Матер_к"] += 1

In [197]:
df.insert(0, 'id', range(1, len(df) + 1))

In [198]:
df = df.rename(columns={"Агресс_ср": "env_aggressiveness_id",
                        "Матер_к": "construction_material_id",
                        "Наим_сп": "name",
                        "Статус_сп": "status"
                       })

In [199]:
cursor.execute(open("sql_server/19.sql", "r").read())
conn.commit()

In [200]:
execute_values(df, "corr_prot_methods")

<h1>Покрытия антикоррозионной защиты</h1>

In [201]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Покрытия_аз', db_conn)

In [202]:
df = df.replace("", None)

In [203]:
df["Тип_лп"] = df["Тип_лп"].str.strip()
df["Стойкость_лп"] = df["Стойкость_лп"].str.strip()
df["Наим_лп"] = df["Наим_лп"].str.strip()

In [204]:
df["Приоритет"] = df["Приоритет"].map(lambda x: 0 if pd.isna(x) else x)

In [205]:
df = df.drop_duplicates(subset=["Тип_лп", "Группа_лп", "Стойкость_лп", "Наим_лп"])

In [206]:
paintwork = pd.read_csv("paintwork.csv")

In [207]:
for index, row in df.iterrows():
    try:
        df.loc[index, "paintwork_type_id"] = paintwork.loc[paintwork["name"] == row["Тип_лп"], "id"].values[0]
    except:
        df.loc[index, "paintwork_type_id"] = None

In [208]:
paintwork_fastness = pd.read_csv("paintwork_fastness.csv")

In [209]:
for index, row in df.iterrows():
    try:
        df.loc[index, "paintwork_fastness_id"] = paintwork_fastness.loc[paintwork_fastness["name"] == row["Стойкость_лп"], "id"].values[0]
    except:
        df.loc[index, "paintwork_fastness_id"] = None

In [210]:
df = df.drop(["Тип_лп", "Стойкость_лп"], axis=1)

In [211]:
df = df.rename(columns={"Покрытие": "id",
                        "Группа_лп": "paintwork_group",
                        "Наим_лп": "name",
                        "Кол_сл_лп": "paintwork_num_of_layers",
                        "Группа_гр": "primer_group",
                        "Можно_лп": "can_be_painted",
                        "Приоритет": "priority"
                       })

In [212]:
df = df.where(pd.notnull(df), None)

In [213]:
cursor.execute(open("sql_server/17.sql", "r").read())
conn.commit()

In [214]:
execute_values(df, "corr_prot_coatings")

<h1>Виды конструкций</h1>

In [215]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_видов_к', db_conn)

In [216]:
df["Имя_к"] = df["Имя_к"].str.strip()

In [217]:
df = df[df["Имя_к"].isna() == False]

In [218]:
df = df.rename(columns={"Шифр_к": "id",
                        "Имя_к": "name"
                       })

In [219]:
df.loc[df["id"] == 2, "name"] = "Двухветвевые колонны одноэтажных производственных зданий и крановых эстакад"
df.loc[df["id"] == 9, "name"] = "Подкраново-подстропильные фермы"
df.loc[df["id"] == 34, "name"] = "Переплеты, витражи из профильных труб с деталями крепления"

In [220]:
cursor.execute(open("sql_server/22.sql", "r").read())
conn.commit()

In [221]:
execute_values(df, "construction_types")

<h1>Подвиды конструкций</h1>

In [222]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_видов_пк', db_conn)

In [223]:
df["Имя_пк"] = df["Имя_пк"].str.strip()
df["Расценка"] = df["Расценка"].str.strip()

In [224]:
df = df.drop_duplicates()

In [225]:
df = df.drop(["Шифр_пк"], axis=1)

In [226]:
df = df.rename(columns={"Поз_пк": "id",
                        "Шифр_к": "type_id",
                        "Имя_пк": "name",
                        "Расценка": "valuation"
                       })

In [227]:
df.loc[df["id"] == 11, "name"] = "высотой свыше 150 м, поставляемые отдельными элементами"
df.loc[df["id"] == 14, "name"] = "коробчатого сечения пролетом 12 м и более"
df.loc[df["id"] == 33, "name"] = "с шагом стропильных ферм 6 м"
df.loc[df["id"] == 34, "name"] = "с шагом стропильных ферм 12 м"
df.loc[df["id"] == 49, "name"] = "составного сечения из горячекатаных профилей, соединенных планками"
df.loc[df["id"] == 51, "name"] = "ветви из двутавров с парал. гранями полок, решетка из г/кат. уголков и швеллеров"
df.loc[df["id"] == 52, "name"] = "ветви из двутавров (ГОСТ 8239-72) или швеллеров и решетка из г/кат. уголков"
df.loc[df["id"] == 69, "name"] = "с поясами из тавров, стенкой из листовой стали, пролетом 6 м, массой 1 м,т:"
df.loc[df["id"] == 70, "name"] = "с поясами из тавров, стенкой из листовой стали, пролетом 12 м и более,т:"
df.loc[df["id"] == 71, "name"] = "из листовой стали пролетом 6 м, массой 1 м,т:"
df.loc[df["id"] == 72, "name"] = "из листовой стали пролетом 12 м, массой 1 м,т:"
df.loc[df["id"] == 83, "name"] = "пролетом 12 м, решетчатые из г/кат. и гнутых профилей, масса 1 м до 0.4 т"
df.loc[df["id"] == 84, "name"] = "пролетом 12 м, решетчатые из г/кат. и гнутых профилей, масса 1 м свыше 0.4 т"
df.loc[df["id"] == 85, "name"] = "пролетом 12 м, решетчатые сост. сечения из листовой стали, масса 1 м до 0.4 т"
df.loc[df["id"] == 86, "name"] = "пролетом 12 м, решетчатые сост. сечения из листовой стали, масса 1 м свыше 0.4 т"
df.loc[df["id"] == 87, "name"] = "составного сечения из листовой стали, масса 1 м,т:"
df.loc[df["id"] == 88, "name"] = "составного сечения из двух и более уголков или швеллеров, масса 1 м,т:"
df.loc[df["id"] == 89, "name"] = "из двутавров с параллельными гранями полок, масса 1 м,т:"
df.loc[df["id"] == 90, "name"] = "составного сечения из листовой стали с верхним опиранием настила, масса марки,т:"
df.loc[df["id"] == 91, "name"] = "составного сечения при пониженном опирании настила, масса марки,т:"
df.loc[df["id"] == 92, "name"] = "из двутавров с парал. гранями полок с верхним опиранием настила, масса до 1 т"
df.loc[df["id"] == 93, "name"] = "из двутавров с парал. гранями полок с верхним опиранием настила, масса свыше 1 т"
df.loc[df["id"] == 94, "name"] = "из двутавров с парал. гранями полок с пониж. опиранием настила, масса до 1 т"
df.loc[df["id"] == 95, "name"] = "из двутавров с парал. гранями полок с пониж. опиранием настила, масса свыше 1 т"
df.loc[df["id"] == 113, "name"] = "фермы из парных уголков и тавров, попер. балки из двутавров, масса 1 м до 0.5 т"
df.loc[df["id"] == 114, "name"] = "фермы из парных уголков и тавров, попер. балки из двутавров, масса 1 м до 1 т"
df.loc[df["id"] == 115, "name"] = "фермы из парных уголков и тавров, попер. балки из двутавров, масса 1 м свыше 1 т"

In [228]:
cursor.execute(open("sql_server/24.sql", "r").read())
conn.commit()

In [229]:
execute_values(df, "construction_subtypes")

<h1>Типовые наименования листов</h1>

In [230]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_листов_н', db_conn)

In [231]:
df["ш_листа"] = df["ш_листа"].str.strip()

In [232]:
df = df.rename(columns={"ш_номер": "id",
                        "ш_листа": "name"
                       })

In [233]:
cursor.execute(open("sql_server/27.sql", "r").read())
conn.commit()

In [234]:
execute_values(df, "sheet_names")

<h1>Виды ссылочных документов</h1>

In [235]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_ссыл_в', db_conn)

In [236]:
df["Наим_вида"] = df["Наим_вида"].str.strip()
df["Наим_вида"] = df["Наим_вида"].str.capitalize()

In [237]:
df = df.drop_duplicates(subset=["Наим_вида"])

In [238]:
df = df.rename(columns={"Вид_сд": "id",
                        "Наим_вида": "name"
                       })

In [239]:
cursor.execute(open("sql_server/40.sql", "r").read())
conn.commit()

In [240]:
execute_values(df, "linked_doc_types")

<h1>Ссылочные документы</h1>

In [241]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_ссыл', db_conn)

In [242]:
df = df.replace("", np.nan)

In [243]:
df = df.dropna()

In [244]:
df = df.drop_duplicates(subset=["Шифр_сд"])

In [245]:
df = df.reset_index(drop=True)

In [246]:
df["Шифр_сд"] = df["Шифр_сд"].str.strip()
df["Обозн_сд"] = df["Обозн_сд"].str.strip()
df["Наим_сд"] = df["Наим_сд"].str.strip()

In [247]:
for i in range(0, len(df)):
    arr = df["Наим_сд"][i].split()
    new_str = ' '.join(arr)
    df.at[i, "Наим_сд"] = new_str

In [248]:
df["Наим_сд"] = df["Наим_сд"].str.replace("возду ха", "воздуха")
df["Наим_сд"] = df["Наим_сд"].str.replace("Материриалы", "Материалы")
df["Наим_сд"] = df["Наим_сд"].str.replace("рекоментации", "рекомендации")
df["Наим_сд"] = df["Наим_сд"].str.replace("рас пашные", "распашные")
df["Наим_сд"] = df["Наим_сд"].str.replace("металличе ческих", "металлических")
df["Наим_сд"] = df["Наим_сд"].str.replace("металличе ских", "металлических")
df["Наим_сд"] = df["Наим_сд"].str.replace("электри ческие", "электрические")
df["Наим_сд"] = df["Наим_сд"].str.replace("кра ны", "краны")
df["Наим_сд"] = df["Наим_сд"].str.replace("примене нием", "применением")
df["Наим_сд"] = df["Наим_сд"].str.replace("обслуживани сосудов", "обслуживания сосудов")
df["Наим_сд"] = df["Наим_сд"].str.replace("пе реплетами", "переплетами")
df["Наим_сд"] = df["Наим_сд"].str.replace("за полнением", "заполнением")
df["Наим_сд"] = df["Наим_сд"].str.replace("линй", "линий")
df["Наим_сд"] = df["Наим_сд"].str.replace("мосто вые", "мостовые")
df["Наим_сд"] = df["Наим_сд"].str.replace("покрыти ем", "покрытием")
df["Наим_сд"] = df["Наим_сд"].str.replace("со кращенному", "сокращенному")
df["Наим_сд"] = df["Наим_сд"].str.replace("покрытияпролетами", "покрытия пролетами")
df["Наим_сд"] = df["Наим_сд"].str.replace("широкополочныых", "широкополочных")
df["Наим_сд"] = df["Наим_сд"].str.replace("тав ров", "тавров")
df["Наим_сд"] = df["Наим_сд"].str.replace("зда ний", "зданий")
df["Наим_сд"] = df["Наим_сд"].str.replace("ремонтно -монтажных", "ремонтно-монтажных")
df["Наим_сд"] = df["Наим_сд"].str.replace("пок рытия", "покрытия")
df["Наим_сд"] = df["Наим_сд"].str.replace("профилирован ным", "профилированным")
df["Наим_сд"] = df["Наим_сд"].str.replace("стеклои минераловатных", "стекло и минераловатных")
df["Наим_сд"] = df["Наим_сд"].str.replace("окон,дверей", "окон, дверей")
df["Наим_сд"] = df["Наим_сд"].str.replace("пром.зданий", "пром. зданий")
df["Наим_сд"] = df["Наим_сд"].str.replace("строитель ные", "строительные")
df["Наим_сд"] = df["Наим_сд"].str.replace("сталь ные", "стальные")
df["Наим_сд"] = df["Наим_сд"].str.replace("наруж ного", "наружного")
df["Наим_сд"] = df["Наим_сд"].str.replace("ветров.район.", "ветров. район.")
df["Наим_сд"] = df["Наим_сд"].str.replace("выше и ниже -40 град.с.  ше и ниже -40 град.с.", "выше -40 град.с. и ниже -40 град.с.")
df["Наим_сд"] = df["Наим_сд"].str.replace("алю миниевых", "алюминиевых")
df["Наим_сд"] = df["Наим_сд"].str.replace("зданий,промышленных", "зданий, промышленных")
df["Наим_сд"] = df["Наим_сд"].str.replace("предприятий.шарнирные", "предприятий. Шарнирные")
df["Наим_сд"] = df["Наим_сд"].str.replace("связей.Чертежи", "связей. Чертежи")
df["Наим_сд"] = df["Наим_сд"].str.replace("сталь ными", "стальными")
df["Наим_сд"] = df["Наим_сд"].str.replace("унифици рованных", "унифицированных")
df["Наим_сд"] = df["Наим_сд"].str.replace("уз лы", "узлы")
df["Наим_сд"] = df["Наим_сд"].str.replace("железобе тонных", "железобетонных")
df["Наим_сд"] = df["Наим_сд"].str.replace("метал лопроката", "металлопроката")
df["Наим_сд"] = df["Наим_сд"].str.replace("выпу ску", "выпуску")
df["Наим_сд"] = df["Наим_сд"].str.replace("откры вающиеся", "открывающиеся")
df["Наим_сд"] = df["Наим_сд"].str.replace("светоаэра ционных", "светоаэрационных")
df["Наим_сд"] = df["Наим_сд"].str.replace("проектирования.Рабочие", "проектирования. Рабочие")
df["Наим_сд"] = df["Наим_сд"].str.replace("черте жи", "чертежи")
df["Наим_сд"] = df["Наим_сд"].str.replace("опорн.кранами", "опорн. кранами")
df["Наим_сд"] = df["Наим_сд"].str.replace("опорынми кранами", "опорными кранами")
df["Наим_сд"] = df["Наим_сд"].str.replace("несущ.конструкций", "несущ. конструкций")
df["Наим_сд"] = df["Наим_сд"].str.replace("подв.кранами", "подв. кранами")
df["Наим_сд"] = df["Наим_сд"].str.replace("кровлей,оборудованных", "кровлей, оборудованных")
df["Наим_сд"] = df["Наим_сд"].str.replace("одноэт.произв", "одноэт. произв")
df["Наим_сд"] = df["Наим_сд"].str.replace("произв.зданий", "произв. зданий")
df["Наим_сд"] = df["Наим_сд"].str.replace("кровлей,оборудованных", "кровлей, оборудованных")
df["Наим_сд"] = df["Наим_сд"].str.replace("специальн.назначения", "специальн. назначения")
df["Наим_сд"] = df["Наим_сд"].str.replace("зданий,оборудованных", "зданий, оборудованных")
df["Наим_сд"] = df["Наим_сд"].str.replace("две рей", "дверей")

In [249]:
df = df.drop_duplicates(subset=["Наим_сд"])

In [250]:
df.loc[df["Наим_сд"].str.endswith('.') == False, "Наим_сд"] = df[df["Наим_сд"].str.endswith('.') == False]["Наим_сд"] + '.'

In [251]:
df.insert(0, 'id', range(1, len(df) + 1))

In [252]:
df = df.rename(columns={"Шифр_сд": "code",
                        "Вид_сд": "type_id",
                        "Обозн_сд": "designation",
                        "Наим_сд": "name"
                       })

In [253]:
cursor.execute(open("sql_server/39.sql", "r").read())
conn.commit()

In [254]:
execute_values(df, "linked_docs")

<h1>Категории производства по пожарной и взрывопожарной опасности</h1>

In [255]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Категории_пр', db_conn)

In [256]:
df = df.replace("", np.nan)

In [257]:
df = df.rename(columns={"поз": "id",
                        "Категория": "category",
                        "Название": "name",
                        "Текст_од": "description"
                       })

In [258]:
df = df.where(pd.notnull(df), None)

In [259]:
cursor.execute(open("sql_server/5.sql", "r").read())
conn.commit()

In [260]:
execute_values(df, "fire_hazard_categories")

<h1>Марки</h1>

In [261]:
employee_df = pd.read_sql_query('SELECT * FROM DocumentsKM.dbo.Employees', conn)

In [262]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Марки', db_conn)

In [263]:
df = df.loc[:, ["обозн", "код_марки", "объект", "комплекс", "гл_спец", "рук_гр", "н_контр", "дата_выд", 'дата_ред', "кол_томов", "прим", 'коэф_надежн', 't_эксплуат', 'зона_эксплуат', 'группа_газов', 'агрессивность', 'материал', 'тип_ЛКМ', 'вп_болты', 'категория_пр', 'ТекстЗдСм', 'ДопОбъемы', "подп1"]]

In [264]:
df["обозн"] = df["обозн"].str.strip()

In [265]:
df["обозн"] = df["обозн"].str.replace("KM", "КМ")

In [266]:
df2 = pd.DataFrame(df["обозн"].str.split('-КМ', 1).tolist(),
                   columns = ['first','code'])

In [267]:
df = df.join(df2[["code"]])

In [268]:
df = df[df["code"] != ""]

In [269]:
df.loc[:, "code"] = "КМ" + df["code"]

In [270]:
df = df.dropna(subset=["кол_томов", "t_эксплуат", "группа_газов", "code", "подп1"])

In [271]:
df["t_эксплуат"] = df["t_эксплуат"].str.replace("минус ", "-")
df["t_эксплуат"] = df["t_эксплуат"].str.replace("плюс ", "")
df["t_эксплуат"] = df["t_эксплуат"].str.replace("+", "")
df["t_эксплуат"] = df["t_эксплуат"].str.replace("- 39", "-39")
df = df[df["t_эксплуат"] != "-"]
df = df[df["t_эксплуат"] != "минус"]
df["t_эксплуат"] = df["t_эксплуат"].astype("float64")

In [272]:
df["прим"] = df["прим"].str.strip()
df["прим"] = df["прим"].str.replace("\r\n       ", " ")

In [273]:
df["категория_пр"] = 3

In [274]:
df["агрессивность"] += 1
df["группа_газов"] += 1
df["материал"] += 1
df["вп_болты"] += 1

In [275]:
df.loc[df["тип_ЛКМ"] == "ПФ", "тип_ЛКМ"] = 1
df.loc[df["тип_ЛКМ"] == "АУ", "тип_ЛКМ"] = 2
df.loc[df["тип_ЛКМ"] == "ЭЭ", "тип_ЛКМ"] = 3
df.loc[df["тип_ЛКМ"] == "МА", "тип_ЛКМ"] = 4
df.loc[df["тип_ЛКМ"] == "ПВ", "тип_ЛКМ"] = 11

In [276]:
df.loc[:, 'подп1'] = to_new_employee_ids(df, 'подп1')
df.loc[:, 'гл_спец'] = to_new_employee_ids(df, 'гл_спец')
df.loc[:, 'рук_гр'] = to_new_employee_ids(df, 'рук_гр')
df.loc[:, 'н_контр'] = to_new_employee_ids(df, 'н_контр')

In [277]:
df = df[(df["гл_спец"].isna() == False) | (df["рук_гр"].isna() == False) | (df["н_контр"].isna() == False)]

In [278]:
for index, row in df.iterrows():
    try:
        v1 = employee_df.loc[employee_df["Id"] == row["гл_спец"], "DepartmentId"].values
        if (len(v1) == 1):
            df.loc[index, "department_id"] = v1[0]
            continue
            
        v1 = employee_df.loc[employee_df["Id"] == row["рук_гр"], "DepartmentId"].values
        if (len(v1) == 1):
            df.loc[index, "department_id"] = v1[0]
            continue
            
        v1 = employee_df.loc[employee_df["Id"] == row["н_контр"], "DepartmentId"].values
        df.loc[index, "department_id"] = v1[0]
    except:
        df.loc[index, "department_id"] = None

In [279]:
df = df.dropna(subset=["department_id"])

In [280]:
df = df.dropna(subset=["объект", "комплекс"])

In [281]:
for index, row in df.iterrows():
    try:
        df.loc[index, "subnode_id"] = mark_archive.loc[mark_archive["Id"] == row["код_марки"], "ArchSubnodeId"].values[0]
    except:
        df.loc[index, "subnode_id"] = None

In [282]:
df = df.dropna(subset=["subnode_id"])

In [283]:
for index, row in df.iterrows():
    try:
        df.loc[index, "ArchSubnodeId"] = mark_archive.loc[mark_archive["Id"] == row["код_марки"], "ArchSubnodeId"].values[0]
    except:
        df.loc[index, "ArchSubnodeId"] = None

In [284]:
for index, row in df.iterrows():
    try:
        df.loc[index, "name"] = archive_marks.loc[archive_marks["Марка"] == row["ArchSubnodeId"], "НазвМарки"].values[0]
    except:
        df.loc[index, "name"] = None

In [285]:
for index, row in df.iterrows():
    try:
        df.loc[index, "node_id"] = archive_subnodes.loc[archive_subnodes["Подузел"] == row["subnode_id"], "Узел"].values[0]
    except:
        df.loc[index, "node_id"] = None

In [286]:
for index, row in df.iterrows():
    try:
        df.loc[index, "chief_engineer_name"] = archive_nodes.loc[archive_nodes["Узел"] == row["node_id"], "ГИП"].values[0]
    except:
        df.loc[index, "chief_engineer_name"] = None

In [287]:
df = df.drop(["ArchSubnodeId", "node_id"], axis = 1)

In [288]:
df = df.drop_duplicates(subset=["code", "subnode_id"])

In [289]:
df_op_cond = df.loc[:, ["код_марки", "коэф_надежн", "агрессивность", "t_эксплуат", "зона_эксплуат", "группа_газов", "материал", "тип_ЛКМ", "вп_болты"]]

In [290]:
df_task = df.loc[:, ["обозн", "код_марки", "ТекстЗдСм", "ДопОбъемы"]]

In [291]:
df = df.drop(["ТекстЗдСм", "ДопОбъемы", "коэф_надежн", "агрессивность", "t_эксплуат", "зона_эксплуат", "группа_газов", "материал", "вп_болты", "тип_ЛКМ"], axis=1)

In [292]:
df = df.rename(columns={"код_марки": "id",
                        "обозн": "designation",
                        "комплекс": "complex_name",
                        "объект": "object_name",
                        "подп1": "signed_id",
                        "гл_спец": "chief_specialist_id",
                        "рук_гр": "group_leader_id",
                        "н_контр": "norm_contr_id",
                        "дата_выд": "issue_date",
                        "кол_томов": "num_of_volumes",
                        "дата_ред": "edited_date",
                        "прим": "note",
                        "категория_пр": "fire_hazard_category_id",
                       })

In [293]:
df["issue_date"] = df["issue_date"].astype(object).where(df["issue_date"].notnull(), None)
df["edited_date"] = df["edited_date"].astype(object).where(df["edited_date"].notnull(), None)
df = df.where(pd.notnull(df), None)

In [294]:
df = df.reset_index(drop=True)

In [295]:
df["id"].to_csv("mark_ids.csv", index=False)

In [296]:
cursor.execute(open("sql_server/7.sql", "r").read())
conn.commit()

In [297]:
execute_values(df, "marks")

<h1>Условия эксплуатации марки</h1>

In [298]:
df_op_cond = df_op_cond.rename(columns={"код_марки": "mark_id",
                            "коэф_надежн": "safety_coeff",
                            "t_эксплуат": "temperature",
                            "зона_эксплуат": "operating_area_id",
                            "группа_газов": "gas_group_id",
                            "агрессивность": "env_aggressiveness_id",
                            "материал": "construction_material_id",
                            "тип_ЛКМ": "paintwork_type_id",
                            "вп_болты": "high_tensile_bolts_type_id"
                       })

In [299]:
cursor.execute(open("sql_server/y_mark_operating_conditions.sql", "r").read())
conn.commit()

In [300]:
execute_values(df_op_cond, "mark_operating_conditions", False)

<h1>Задание на смету</h1>

In [301]:
df_task = df_task.rename(columns={"код_марки": "mark_id",
                                  "ТекстЗдСм": "task_text",
                                  "ДопОбъемы": "additional_text",
                                 })

In [302]:
df_task["task_text"] = df_task["task_text"].str.strip()
df_task["additional_text"] = df_task["additional_text"].str.strip()

In [303]:
df_task["task_text"] = df_task["task_text"].str.replace("\r\n", "\n")
df_task["additional_text"] = df_task["additional_text"].str.replace("\r\n", "\n")

In [304]:
df_task['обозн'] = 'Разработать сметную документацию к чертежам ' + df_task['обозн'].astype(str) + '\r\nСостав и объемы работ:'

In [305]:
df_task.loc[df_task["task_text"].isna(), "task_text"] = df_task.loc[df_task["task_text"].isna(), "обозн"]

In [306]:
df_task = df_task.drop(["обозн"], axis=1)

In [307]:
df_task['additional_text'] = df_task['additional_text'].str.replace('град.С', '°C')

In [308]:
cursor.execute(open("sql_server/y_estimate_task.sql", "r").read())
conn.commit()

In [309]:
execute_values(df_task, "estimate_task", False)

<h1>Выпуски спецификаций</h1>

In [310]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.ВыпСпец', db_conn)

In [311]:
df = df.replace("", np.nan)

In [312]:
df = df.drop_duplicates(subset=["код_марки", "выпуск"])

In [313]:
df = df.drop(["вкл_вып"], axis=1)

In [314]:
df["текущий"] = df["текущий"].map(lambda x: True if (x == 1) else False)

In [315]:
df = df.rename(columns={"Поз_выпуска": "id",
                        "код_марки": "mark_id",
                        "выпуск": "num",
                        "дата_созд": "created_date",
                        "текущий": "is_current",
                        "прим": "note"
                       })

In [316]:
df["created_date"] = df["created_date"].astype(object).where(df["created_date"].notnull(), datetime(2000, 1, 1))
df = df.where(pd.notnull(df), None)

In [317]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.values.flatten().tolist()

In [318]:
df = df[df["mark_id"].isin(m_ids)]

In [319]:
df.loc[:, ["id", "mark_id", "num"]].to_csv("specs.csv", index=False)

In [320]:
cursor.execute(open("sql_server/3.sql", "r").read())
conn.commit()

In [321]:
execute_values(df, "specifications")

<h1>Листы</h1>

In [322]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Листы', db_conn)

In [323]:
df = df.replace("", np.nan)

In [324]:
df = df[df["номер"].isna() == False]

In [325]:
df["тип_док"] = df["тип_док"] + 1

In [326]:
df["вып"] = df["вып"].map(lambda x: np.nan if x == 0 else x)

In [327]:
df["пров"] = df["пров"].map(lambda x: np.nan if x == 0 else x)

In [328]:
df["норм"] = df["норм"].map(lambda x: np.nan if x == 0 else x)

In [329]:
df = df.drop(["шифр_вып", "шифр_пров", "шифр_норм"], axis=1)

In [330]:
df["формат"] = df["формат"].map(lambda x: 1.0 if np.isnan(x) else x)

In [331]:
df = df.drop_duplicates(subset=["код_марки", "номер", "тип_док"])

In [332]:
df = df[df["вып"].isna() == False]

In [333]:
df["название"] = df["название"].str.strip()

In [334]:
for i in range(0, len(df)):
    try:
        arr = df["название"][i].split()
        new_str = ' '.join(arr)
        df.at[i, "название"] = new_str
    except Exception:
        pass

In [335]:
df.loc[:, 'вып'] = to_new_employee_ids(df, 'вып')
df.loc[:, 'пров'] = to_new_employee_ids(df, 'пров')
df.loc[:, 'норм'] = to_new_employee_ids(df, 'норм')

In [336]:
df = df.rename(columns={"поз_листа": "id",
                        "код_марки": "mark_id",
                        "номер": "num",
                        "название": "name",
                        "формат": "form",
                        "вып": "creator_id",
                        "пров": "inspector_id",
                        "норм": "norm_contr_id",
                        "тип_док": "type_id",
                        "выпуск": "release_num",
                        "листов": "num_of_pages",
                        "прим": "note"
                       })

In [337]:
df = df.where(pd.notnull(df), None)

In [338]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.values.flatten().tolist()

In [339]:
e_ids = pd.read_csv("employee_ids.csv")
e_ids = e_ids.values.flatten().tolist()

In [340]:
df = df[df["mark_id"].isin(m_ids)]
df.loc[df["creator_id"].isin(e_ids) == False, "creator_id"] = None
df.loc[df["inspector_id"].isin(e_ids) == False, "inspector_id"] = None
df.loc[df["norm_contr_id"].isin(e_ids) == False, "norm_contr_id"] = None

In [341]:
df["release_num"] = df["release_num"].fillna(0)
df["num_of_pages"] = df["num_of_pages"].fillna(0)

In [342]:
df = df[df["creator_id"].isna() == False]
df = df[df["name"].isna() == False]

In [343]:
cursor.execute(open("sql_server/6.sql", "r").read())
conn.commit()

In [344]:
execute_values(df, "docs")

<h1>Обозначения марок</h1>

In [345]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Об_марок', db_conn)

In [346]:
df = df.iloc[:-1, :]

In [347]:
df.loc[0, "об_марки"] = "КМД"
df.loc[0, "наим_марки"] = "Конструкции металлические деталировочные"

In [348]:
df = df.rename(columns={"Поз": "id",
                        "об_марки": "designation",
                        "наим_марки": "name"
                       })

In [349]:
cursor.execute(open("sql_server/9.sql", "r").read())
conn.commit()

In [350]:
execute_values(df, "mark_designations")

<h1>Прилагаемые документы марки</h1>

In [351]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Од_прил', db_conn)

In [352]:
df = df.replace("", np.nan)

In [353]:
df = df.drop_duplicates(subset=["код_марки", "обозн_прил"])

In [354]:
df = df.where(pd.notnull(df), None)

In [355]:
df = df.rename(columns={"Поз_прил": "id",
                        "код_марки": "mark_id",
                        "наим_прил": "name",
                        "прим": "note",
                        "обозн_прил": "designation"
                       })

In [356]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.values.flatten().tolist()
df = df[df["mark_id"].isin(m_ids)]

In [357]:
cursor.execute(open("sql_server/12.sql", "r").read())
conn.commit()

In [358]:
execute_values(df, "attached_docs")

<h1>Согласования</h1>

In [359]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Од_согл', db_conn)

In [360]:
df = df.replace("", np.nan)

In [361]:
df = df.drop(["отдел1", "отдел2", "отдел3", "отдел4", "отдел5", "отдел6", "отдел7"], axis=1)

In [362]:
p1 = df.iloc[:, [0,1]]
p1 = p1.rename(columns={"исп1": "исп"})

p2 = df.iloc[:, [0,2]]
p2 = p2.rename(columns={"исп2": "исп"})

p3 = df.iloc[:, [0,3]]
p3 = p3.rename(columns={"исп3": "исп"})

p4 = df.iloc[:, [0,4]]
p4 = p4.rename(columns={"исп4": "исп"})

p5 = df.iloc[:, [0,5]]
p5 = p5.rename(columns={"исп5": "исп"})

p6 = df.iloc[:, [0,6]]
p6 = p6.rename(columns={"исп6": "исп"})

p7 = df.iloc[:, [0,7]]
p7 = p7.rename(columns={"исп7": "исп"})

In [363]:
df = pd.concat([p1, p2, p3, p4, p5, p6, p7], ignore_index=True)

In [364]:
df = df.dropna()
df = df[df["исп"] != 0]

In [365]:
df.insert(0, 'id', range(1, len(df) + 1))

In [366]:
df.loc[:, 'исп'] = to_new_employee_ids(df, 'исп')

In [367]:
df = df.drop_duplicates(subset=["код_марки", "исп"])

In [368]:
df = df.rename(columns={"код_марки": "mark_id",
                        "исп": "employee_id"
                       })

In [369]:
m_ids = pd.read_csv("mark_ids.csv")
e_ids = pd.read_csv("employee_ids.csv")

In [370]:
m_ids = m_ids.values.flatten().tolist()
e_ids = e_ids.values.flatten().tolist()

In [371]:
df = df[df["mark_id"].isin(m_ids)]
df = df[df["employee_id"].isin(e_ids)]

In [372]:
cursor.execute(open("sql_server/13.sql", "r").read())
conn.commit()

In [373]:
execute_values(df, "mark_approvals")

<h1>Ссылочные документы марки</h1>

In [374]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Од_ссыл', db_conn)

In [375]:
df = df.replace("", np.nan)

In [376]:
df= df[df["шифр_сд"] != "1"]
df["шифр_сд"] = df["шифр_сд"].str.upper()

In [377]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]

In [378]:
for i in range(1, 301):
    df = df.append(pd.Series(), ignore_index=True)

In [379]:
df.loc[:, "код_марки"] = random.choices(m_ids, k=300)
df.loc[:, "шифр_сд"] = random.choices(range(1, 300), k=300)

In [380]:
df = df.drop_duplicates(subset=["код_марки", "шифр_сд"])

In [381]:
df = df.drop(["Поз_сд"], axis=1)

In [382]:
df.insert(0, 'id', range(1, len(df) + 1))

In [383]:
df = df.rename(columns={"код_марки": "mark_id",
                        "шифр_сд": "linked_doc_id",
                        "прим": "note"
                       })

In [384]:
cursor.execute(open("sql_server/14.sql", "r").read())
conn.commit()

In [385]:
execute_values(df, "mark_linked_docs")

<h1>Учет дополнительных работ</h1>

In [386]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Од_вып_доп', db_conn)

In [387]:
df = df.replace("", np.nan)

In [388]:
df = df.drop(["шифр_исп"], axis=1)

In [389]:
df = df[df["код_исп"].isna() == False]

In [390]:
df.loc[:, 'код_исп'] = to_new_employee_ids(df, 'код_исп')

In [391]:
df = df.drop_duplicates(subset=["код_марки", "код_исп"])

In [392]:
df = df.dropna(subset=["код_исп"])

In [393]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]

In [394]:
df = df.rename(columns={"Поз_вып_доп": "id",
                        "код_марки": "mark_id",
                        "код_исп": "employee_id",
                        "расчет": "valuation",
                        "заказ": "metal_order",
                       })

In [395]:
cursor.execute(open("sql_server/11.sql", "r").read())
conn.commit()

In [396]:
execute_values(df, "additional_work")

<h1>Типовые конструкции</h1>

In [397]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Тс_типовые', db_conn)

In [398]:
df = df[df["имя_тк"].isna() == False]

In [399]:
df["имя_тк"] = df["имя_тк"].str.strip()

<h2>Типовые наименования типовых конструкций</h2>

In [400]:
st_name_df = pd.DataFrame(data=df["имя_тк"].unique(), columns=["name"])

In [401]:
st_name_df.loc[3, "name"] = "Рельсы крановые КР-70 сталь К63"
st_name_df = st_name_df[st_name_df["name"] != "КР-70"]
st_name_df.loc[5, "name"] = "Рельсы крановые Р-43"
st_name_df = st_name_df[st_name_df["name"] != "Оцинкованная сталь ГОСТ14918-80*"]
st_name_df.loc[11, "name"] = "Рельсы крановые КР-70"
st_name_df.loc[12, "name"] = "Детали крепления крановых рельсов"
st_name_df = st_name_df[st_name_df["name"] != "Крановый рельс КР-70"]
st_name_df.loc[16, "name"] = "Накладка 5 на рельс Р-24 по ГОСТ 8141-56"

In [402]:
st_name_df.insert(0, 'id', range(1, len(st_name_df) + 1))

In [403]:
cursor.execute(open("sql_server/yyy_1.sql", "r").read())
conn.commit()

In [404]:
execute_values(st_name_df, "standard_construction_names")

--------------------

In [405]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]

In [406]:
specs = pd.read_csv("specs.csv")

In [407]:
for index, row in df.iterrows():
    try:
        df.loc[index, "specification_id"] = specs.loc[(specs["mark_id"] == row["код_марки"]) & (specs["num"] == row["выпуск"]), "id"].values[0]
    except:
        pass

In [408]:
df = df.drop(["код_марки", "выпуск"], axis=1)

In [409]:
if 24 in df.index:
    df.loc[24, "кол_тк"] = 1

In [410]:
# https://wikiweight.info/strmat/massa-ankerov-hilti.html
if 28 in df.index:
    df.loc[28, "вес_тк"] = 0.056
if 32 in df.index:
    df.loc[32, "вес_тк"] = 0.056

In [411]:
if 25 in df.index:
    df.loc[25, "вес_тк"] = 0.024
if 29 in df.index:
    df.loc[29, "вес_тк"] = 0.024
if 25 in df.index:
    df.loc[25, "кол_тк"] = 360
if 29 in df.index:
    df.loc[29, "кол_тк"] = 360
df = df[df["сч_тк"] != 54]
df = df[df["сч_тк"] != 58]

In [412]:
df = df.rename(columns={"сч_тк": "id",
                        "имя_тк": "name",
                        "кол_тк": "num",
                        "лист_тк": "sheet",
                        "вес_тк": "weight"
                       })

In [413]:
df = df.where(pd.notnull(df), None)

In [414]:
cursor.execute(open("sql_server/47.sql", "r").read())
conn.commit()

In [415]:
execute_values(df, "standard_constructions")

<h1>Виды конструкций марки</h1>

In [416]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Тс_Виды_к', db_conn)

In [417]:
df = df.drop_duplicates()

In [418]:
df = df.dropna(subset=["кт_окраски", "кол_тип_констр", "имя_к", "Шифр_к", "Шифр_пк"])

In [419]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]

In [420]:
specs = pd.read_csv("specs.csv")

In [421]:
df.insert(1, "specification_id", 0, True) 

In [422]:
for index, row in df.iterrows():
    try:
        df.loc[index, "specification_id"] = specs.loc[(specs["mark_id"] == row["код_марки"]) & (specs["num"] == row["выпуск"]), "id"].values[0]
    except:
        pass

In [423]:
df = df[df["specification_id"] != 0]

In [424]:
df = df.drop(["код_марки", "выпуск"], axis=1)

In [425]:
df["Шифр_пк"] = df["Шифр_пк"].replace(0, np.nan)

In [426]:
df = df.drop(["вкл", "окраска", "огнезащита", "ПределОС", "ПредСост_несущ", "ПредСост_целост", "ПредСост_тепло_изол", "ПредСост_тепло_поток", "ПредСост_газ", "статус_к", "ТеорРасходОЗ"], axis=1)

In [427]:
df = df.rename(columns={"сч_констр": "id",
                        "имя_к": "name",
                        "Шифр_к": "type_id",
                        "Шифр_пк": "subtype_id",
                        "расценка": "valuation",
                        "шифр_тип_альбома": "standard_album_code",
                        "кол_тип_констр": "num_of_standard_constructions",
                        "притупление": "has_edge_blunting",
                        "динамика": "has_dynamic_load",
                        "фланцы": "has_flanged_connections",
                        "контроль_сварки": "welding_control_id",
                        "кт_окраски": "paintwork_coeff",
                       })

In [428]:
df = df.where(pd.notnull(df), None)

In [429]:
df = df[df["type_id"] < 88]

In [430]:
df = df[df["type_id"] > 0]

In [431]:
df = df.drop_duplicates(subset=["specification_id", "name", "paintwork_coeff"])

In [432]:
df.loc[:, ["id"]].to_csv("construction_ids.csv", index=False)

In [433]:
cursor.execute(open("sql_server/46.sql", "r").read())
conn.commit()

In [434]:
execute_values(df, "constructions")

<h1>Справочник болтов, диаметры</h1>

In [435]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_болтов', db_conn)

In [436]:
df = df.rename(columns={"код_д": "id",
                        "диаметр": "diameter",
                        "масса_г": "nut_weight",
                        "сталь_ш": "washer_steel",
                        "масса_ш": "washer_weight",
                        "толщина_ш": "washer_thickness",
                        "ТУ_б": "bolt_tech_spec",
                        "кл_пр_б": "strength_class",
                        "ТУ_г": "nut_tech_spec",
                        "ТУ_ш": "washer_tech_spec",
                       })

In [437]:
cursor.execute(open("sql_server/20.sql", "r").read())
conn.commit()

In [438]:
execute_values(df, "bolt_diameters")

<h1>Справочник болтов, длины</h1>

In [439]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_болтов_дл', db_conn)

In [440]:
df = df.rename(columns={"Код_болта": "id",
                        "код_д": "diameter_id",
                        "длина_б": "length",
                        "длина_нар": "screw_length",
                        "масса_б": "weight"
                       })

In [441]:
cursor.execute(open("sql_server/21.sql", "r").read())
conn.commit()

In [442]:
execute_values(df, "bolt_lengths")

<h1>Высокопрочные болты видов конструкций</h1>

In [443]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Тс_болты', db_conn)

In [444]:
c_ids = pd.read_csv("construction_ids.csv")
c_ids = c_ids.values.flatten().tolist()

In [445]:
df = df[df["сч_констр"].isin(c_ids)]

In [446]:
df.loc[:, "код_диаметра"] = df.loc[:, "код_диаметра"].replace(0, np.nan)

In [447]:
df = df.dropna()

In [448]:
df = df.rename(columns={"поз_болта": "id",
                        "сч_констр": "construction_id",
                        "код_диаметра": "diameter_id",
                        "пакет": "packet",
                        "кол": "num",
                        "гаек": "nut_num",
                        "шайб": "washer_num",
                       })

In [449]:
cursor.execute(open("sql_server/45.sql", "r").read())
conn.commit()

In [450]:
execute_values(df, "construction_bolts")

<h1>Виды профилей</h1>

In [451]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_видов_п', db_conn)

In [452]:
df["Имя_в"] = df["Имя_в"].str.strip()
df["Прим"] = df["Прим"].str.strip()

In [453]:
df = df.drop_duplicates(subset=["Имя_в"])

In [454]:
df = df.rename(columns={"Шифр_в": "id",
                        "Имя_в": "name",
                        "Прим": "note"
                       })

In [455]:
df = df.sort_values(by=["id"])

In [456]:
df = df.reset_index(drop=True)

In [457]:
df.loc[:, ["id"]].to_csv("prof.csv", index=False)

In [458]:
cursor.execute(open("sql_server/23.sql", "r").read())
conn.commit()

In [459]:
execute_values(df, "profile_classes")

<h1>Типы профилей</h1>

In [460]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_типов_проф', db_conn)

In [461]:
df["Имя_т"] = df["Имя_т"].str.strip()

In [462]:
df = df.rename(columns={"Тип": "id",
                        "Имя_т": "name"
                       })

In [463]:
cursor.execute(open("sql_server/42.sql", "r").read())
conn.commit()

In [464]:
execute_values(df, "profile_types")

<h1>Профили</h1>

In [465]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_проф_полн', db_conn)

In [466]:
df = df.replace("", np.nan)

In [467]:
df = df[df["Имя_п"].isna() == False]

In [468]:
df = df[df["Вес"].isna() == False]

In [469]:
df["Имя_п"] = df["Имя_п"].str.strip()
df["Имя_п0"] = df["Имя_п0"].str.strip()

In [470]:
df = df.drop(["Имя_п0", "дл_имя"], axis=1)

In [471]:
df = df[df["Разрешен"]]
df = df.drop(["Разрешен"], axis=1)

In [472]:
df = df[(df["Имя_п"] != "200*160*7") | (df["Символ"].isna() == False)]

In [473]:
df = df.drop_duplicates(subset=["Вид_п", "Имя_п", "Символ"])

In [474]:
df.loc[df["Вид_п"] == 16, "Символ"] = "-"
df.loc[df["Вид_п"] == 17, "Символ"] = "-"

In [475]:
df_2 = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Тс_элементы', db_conn)

In [476]:
df_2["Имя_п"] = df_2["Имя_п"].str.strip()
df_2["Символ"] = df_2["Символ"].str.strip()

In [477]:
c_ids = pd.read_csv("construction_ids.csv")
c_ids = c_ids.values.flatten().tolist()

In [478]:
df_2 = df_2[df_2["Сч_констр"].isin(c_ids)]

In [479]:
p_ids = pd.read_csv("prof.csv")
p_ids = p_ids.values.flatten().tolist()

In [480]:
df_2 = df_2[df_2["Вид_п"].isin(p_ids)]

In [481]:
df_2 = df_2[df_2["Символ"].isna() == False]

In [482]:
df_2 = df_2.drop(["МаркаОЗ", "АрифмВыр"], axis=1)

In [483]:
df_2 = df_2.drop(["ПерекрПерим", "КолВСеч", "Поз", "ГруппаПТ"], axis=1)

In [484]:
for index, row in df_2.iterrows():
    try:
        df_2.loc[index, "profile_id"] = df.loc[(df["Вид_п"] == row["Вид_п"]) & (df["Имя_п"] == row["Имя_п"]) & (df["Символ"] == row["Символ"]), "Профиль"].values[0]
    except Exception as e:
        pass

In [485]:
df_null = df_2.loc[df_2['profile_id'].isna()]

In [486]:
df_null = df_null.drop(["Сч_констр", "Сч_поз", "Сталь", "Длина", "статус", "profile_id"], axis=1)

In [487]:
df_null = df_null.rename(columns={"Вид_п": "class_id",
                        "Имя_п": "name",
                        "Символ": "symbol",
                        "Вес": "weight",
                        "Поверхн": "area",
                        "Тип_п": "type_id"
                       })

In [488]:
df_null.insert(0, 'id', range(1962, len(df_null) + 1962))

In [489]:
df = df.where(pd.notnull(df), None)

In [490]:
df = df.rename(columns={"Профиль": "id",
                        "Вид_п": "class_id",
                        "Имя_п": "name",
                        "Символ": "symbol",
                        "Вес": "weight",
                        "Поверхн": "area",
                        "Тип_п": "type_id"
                       })

In [491]:
df = df.append(df_null, ignore_index = True)

In [492]:
df[df.duplicated(subset=["class_id", "name", "symbol"], keep="first")]
df = df.drop_duplicates(subset=["class_id", "name", "symbol"])

In [493]:
df = df[df["id"] != 3194]

In [494]:
df.loc[df["id"] == 2755, "weight"] = 4.396
df.loc[df["id"] == 2755, "area"] = 0.00288

In [495]:
df.loc[:, ["id", "class_id", "name", "symbol"]].to_csv("profiles.csv", index=False)

In [496]:
cursor.execute(open("sql_server/33.sql", "r").read())
conn.commit()

In [497]:
execute_values(df, "profiles")

<h1>Марки стали</h1>

In [498]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_стали', db_conn)

In [499]:
df = df.replace("", np.nan)

In [500]:
df = df.drop([71])

In [501]:
df["Имя_с"] = df["Имя_с"].str.strip()
df["Стандарт"] = df["Стандарт"].str.strip()

In [502]:
df = df.rename(columns={"Шифр_с": "id",
                        "Имя_с": "name",
                        "Стандарт": "standard",
                        "Прочность": "strength"
                       })

In [503]:
df = df.where(pd.notnull(df), None)

In [504]:
cursor.execute(open("sql_server/41.sql", "r").read())
conn.commit()

In [505]:
execute_values(df, "steel")

<h1>Перечень элементов</h1>

In [506]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Тс_элементы', db_conn)

In [507]:
df["Имя_п"] = df["Имя_п"].str.strip()
df["Символ"] = df["Символ"].str.strip()

In [508]:
c_ids = pd.read_csv("construction_ids.csv")
c_ids = c_ids.values.flatten().tolist()

In [509]:
df = df[df["Сч_констр"].isin(c_ids)]

In [510]:
p_ids = pd.read_csv("prof.csv")
p_ids = p_ids.values.flatten().tolist()

In [511]:
df = df[df["Вид_п"].isin(p_ids)]

In [512]:
df.loc[df["Сталь"] == 72, "Сталь"] = 36

In [513]:
df = df[df["Символ"].isna() == False]

In [514]:
df = df.drop(["МаркаОЗ", "АрифмВыр"], axis=1)

In [515]:
df = df.drop(["ПерекрПерим", "КолВСеч", "Поз", "ГруппаПТ"], axis=1)

In [516]:
profiles = pd.read_csv("profiles.csv")

In [517]:
df.loc[df["Сч_поз"] == 179045, "Имя_п"] = "140*4"
df.loc[df["Сч_поз"] == 179034, "Имя_п"] = "140*4"

In [518]:
for index, row in df.iterrows():
    try:
        df.loc[index, "profile_id"] = profiles.loc[(profiles["class_id"] == row["Вид_п"]) & (profiles["name"] == row["Имя_п"]) & (profiles["symbol"] == row["Символ"]), "id"].values[0]
    except:
        pass

In [519]:
df = df.drop(["Имя_п", "Символ", "Вес", "Поверхн", "Тип_п", "статус", "Вид_п"], axis=1)

In [520]:
df = df.rename(columns={"Сч_констр": "construction_id",
                        "Сч_поз": "id",
                        "Сталь": "steel_id",
                        "Длина": "length"
                       })

In [521]:
cursor.execute(open("sql_server/48.sql", "r").read())
conn.commit()

In [522]:
execute_values(df, "construction_elements")

<h1>Разделы общих указаний пользователя</h1>

In [523]:
df = pd.DataFrame({'name': ["Разработано на основании",
                            "Разработано в соответствии",
                            "Перечень технических регламентов",
                            "Высотная отметка",
                            "Климатические условия",
                            "Конструктивные решения",
                            "Характеристики конструкций",
                            "Материал конструкций",
                            "Указания по разработке КМД и ППР",
                            "Указания по изготовлению и монтажу",
                            "Крепление элементов",
                            "Соединение элементов",
                            "Антикоррозионная защита",
                            "Огнезащита конструкций",
                            "Перечень скрытых видов работ",
                            "Перечень ответственных конструкций",
                            "Коэффициент трудозатрат"]})

In [524]:
df_sections_len = len(df)

In [525]:
df.insert(1, 'order_num', range(1, len(df) + 1))

In [526]:
df2 = df.copy()

In [527]:
df.insert(2, 'user_id', 1)
df2.insert(2, 'user_id', 2)

In [528]:
df = df.append(df2, ignore_index=True)

In [529]:
df.insert(0, 'id', range(1, len(df) + 1))

In [530]:
cursor.execute(open("sql_server/37_2.sql", "r").read())
conn.commit()

In [531]:
execute_values(df, "general_data_sections")

<h1>Пункты и разделы общих указаний марки</h1>

In [532]:
df_sections = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_разделы_н', db_conn)
df_sections_full = df_sections.copy()

In [533]:
df_sections = df_sections.loc[:, ["раздел", "заголовок_кр", "порядок_р"]]

In [534]:
df_sections = df_sections.rename(columns={"раздел": "id",
                        "заголовок_кр": "name",
                        "порядок_р": "order_num",
                       })

In [535]:
df2 = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_пункты_н', db_conn)

In [536]:
df2["текст"] = df2["текст"].str.strip()

In [537]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Оу_пункты_н', db_conn)

In [538]:
df = df[df["включать"] == True]

In [539]:
df["текст_о"] = df["текст_о"].str.strip()

In [540]:
df = df.drop(["включать", "оригинал", "оригинал", "пункт_фикс_о", "нов_пункт", "неразрывать_о"], axis=1)

In [541]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]

In [542]:
df = df[df["пункт_о"].isin(list(df2["пункт"]))]

In [543]:
df3 = pd.DataFrame({'раздел': [], 'текст': [], 'порядок_п': []})
for index, row in df.iterrows():
    df3 = df3.append(df2.loc[df2["пункт"] == df["пункт_о"][index], ["раздел", "текст", "порядок_п"]])

In [544]:
df.reset_index(drop=True, inplace=True)
df3.reset_index(drop=True, inplace=True)
df4 = pd.concat([df, df3], axis=1)

In [545]:
for index, row in df4.iterrows():
    if df4.loc[index, "текст_о"] == None:
        df4.loc[index, "текст_о"] = df4.loc[index, "текст"]

In [546]:
df4 = df4.drop(["пункт_о", "текст"], axis=1)

In [547]:
df4.loc[3, "текст_о"] = '#Конструкции с монтажными сварными соединениями закреплять сначала на временных болтах класса точности С (М16; М20; М24) (выбрать нужное) классов прочности 4.6; 4.8 (выбрать нужное), а затем по проекту.'

In [548]:
df4 = df4.dropna()

In [549]:
df4 = df4.replace("", None)

In [550]:
global_point_num = df4["Поз_пункта"].max() + 1

In [551]:
def changeOrderNumGroupBySection(df, m):
    global global_point_num
    df_list = []
    for s in df["раздел"].unique():
        df2 = df[df["раздел"] == s]
        df2 = df2.sort_values(by=['порядок_п'])
        df3 = df2.drop('текст_о', axis=1).join(df2['текст_о'].str.split('\r\n', expand=True).stack().reset_index(level=1, drop=True).rename('текст_о'))
        df3 = df3[df3["текст_о"].str.len() > 0]
        df3["текст_о"] = df3["текст_о"].map(lambda x: "# " + x[1:] if x[0] == "#" else x)
        df3 = df3.drop_duplicates(subset=['раздел', 'текст_о'])
        df3["порядок_п"] = range(2, len(df3) + 2)
        name = df_sections_full.loc[df_sections_full["раздел"] == s, "заголовок"]
        name = list(name)[0]
        df3["текст_о"] = df3["текст_о"].str.strip()
        df3["текст_о"] = df3["текст_о"].str.replace(r'\b;$', '', regex=True).str.strip()
        df3["текст_о"] = df3["текст_о"].str.replace(r'\b\.$', '', regex=True).str.strip()
        df3 = pd.concat([pd.DataFrame([[global_point_num, m, name, s, 1]], columns=['Поз_пункта', 'код_марки', 'текст_о', 'раздел', 'порядок_п']), df3])
        global_point_num += 1
        df_list.append(df3.copy())
    return pd.concat(df_list)

In [552]:
def changeOrderNumGroupByMark(df):
    df_list = []
    section_df_list = []
    id_num = 1
    for m in df["код_марки"].unique():
        df2 = df[df["код_марки"] == m]
        sections = df2['раздел'].unique()
        df3 = df_sections[df_sections["id"].isin(sections)]
        df3 = df3.rename(columns={"id": "old_id"})
        df3.insert(0, 'id', range(id_num, len(df3) + id_num))
        df3.insert(2, 'mark_id', m)
        df3 = df3.drop(["order_num"], axis=1)
        df3.insert(2, 'order_num', range(1, len(df3) + 1))
        section_df_list.append(df3.copy())
        df2 = changeOrderNumGroupBySection(df2, m)
        df2 = df2.reset_index(drop=True)
        for index, row in df2.iterrows():
            df2.loc[index, "раздел"] = df3.loc[df3["old_id"] == row["раздел"], "id"].values[0]

        df_list.append(df2.copy())
        id_num += len(df3)
        break
    return (pd.concat(df_list), pd.concat(section_df_list))

In [553]:
(df5, df6) = changeOrderNumGroupByMark(df4)

In [554]:
df6 = df6.drop(["old_id"], axis=1)

In [555]:
cursor.execute(open("sql_server/37_3.sql", "r").read())
conn.commit()

In [556]:
execute_values(df6, "mark_general_data_sections")

In [557]:
df5 = df5.drop(["код_марки"], axis=1)

In [558]:
df5 = df5.reset_index(drop=True)

In [559]:
df5['текст_о'] = df5['текст_о'].str.replace('град.С', '°C')
df5['текст_о'] = df5['текст_о'].str.replace('м2', 'м^2')

In [560]:
df5 = df5.rename(columns={"Поз_пункта": "id",
                          "текст_о": "text",
                          "раздел": "section_id",
                          "порядок_п": "order_num",
                       })

In [561]:
df5 = df5.drop(['id'], axis=1)

In [562]:
df5.insert(0, 'id', range(1, len(df5) + 1))

In [563]:
cursor.execute(open("sql_server/35_xx.sql", "r").read())
conn.commit()

In [564]:
execute_values(df5, "mark_general_data_points")

<h1>Пункты общих указаний пользователя</h1>

In [565]:
df = pd.DataFrame({'section_id': [1, 1, 1, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 9, 9, 10, 10, 10, 10, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 15, 15, 15, 15],
                   'text': ["Рабочая документация разработана на основании:",
                            "- поручения от 28.02.2020 № АГЛ-35/0362",
                            "- письма от 05.03.2020 № УПиН-35/0431",
                            "- рекомендаций, выполненного специалистами \"АО МАГНИТОГОРСКИЙ ГИПРОМЕЗ\"",
                            "Рабочая документация разработана в соответствии с техническими условиями, требованиями действующих технических регламентов, стандартов, сводов правил и других документов, содержащих установленные требования.",
                            "Перечень технических регламентов и нормативных документов, в соответствии с требованиями которых разработана рабочая документация",
                            "- Федеральный закон от 30.12.2009 № 384-ФЗ \"Технический регламент о безопасности зданий и сооружений\"",
                            "- Федеральный закон от 22.07.2008 № 123-ФЗ \"Технический регламент о требованиях пожарной безопасности\"",
                            "- СП 16.13330.2017 \"Стальные конструкции. Актуализированная редакция СНиП II-23-81*\"",
                            "- СП 20.13330.2016 \"Нагрузки и воздействия. Актуализированная редакция СНиП 2.01.07-85*\"",
                            "- СП 28.13330.2017 \"Защита строительных конструкций от коррозии. Актуализированная редакция СНиП 2.03.11-85\"",
                            "- СП 53-102-2004 \"Общие правила проектирования стальных конструкций\"",
                            "- СП 70.13330.2012 \"Несущие и ограждающие конструкции. Актуализированная редакция СНиП 3.03.01-87\"",
                            "- ГОСТ 23118-2012 \"Конструкции стальные строительные. Общие технические условия\"",
                            "- ГОСТ 27751-2014 \"Надежность строительных конструкций и оснований. Основные положения\"",
                            "- ГОСТ 9.402-2004 \"Единая система защиты от коррозии и старения (ЕСЗКС). Покрытия лакокрасочные. Подготовка металлических поверхностей к окрашиванию\"",
                            "Высотные отметки абсолютные.",
                            "Климатические условия",
                            "- климатический район – IB",
                            "- расчетная температура наружного воздуха наиболее холодных суток обеспеченностью 0,98 – минус 43 °C",
                            "- нормативное значение веса снегового покрова на 1 м^2 горизонтальной поверхности земли для г. Магнитогорска (согласно СП 20.13330-2016, приложение K, табл. K1) – 1,3 кПа (130 кг/м^2)",
                            "- нормативное значение ветрового давления для II района (согласно СП 20.13330-2016, п. 11.14, табл. 11.1) – 0,30 кПа (30 кг/м^2)",
                            "- зона влажности – сухая",
                            "- расчетная сейсмическая активность – 5 баллов",
                            "Конструктивные решения",
                            "# Чертежи марки \"КМ\" являются исходным материалом для разработки деталировочных чертежей марки \"КМД\"",
                            "# В данной рабочей документации разработано восстановление пролетных строений (усиление ферм пролетных строений, металлоконструкций по верхним и нижним поясам ферм) и восстановление антикоррозионного покрытия существующих элементов пролетных строений галереи конвейера № 706-1 УУК и КДИ аглоцеха ГОП ПАО \"ММК\"",
                            "# Все работы должны выполняться специализированной организацией, имеющей свидетельство СРО на данный вид деятельности и имеющие допуск по электробезопасности",
                            "# Работы производить по специально разработанному и согласованному в установленном порядке со всеми инспектирующими организациями и эксплуатацией цеха проекту производства работ (ППР) с соблюдением требований действующих нормативных документов и обязательным учетом фактического состояния существующих конструкций",
                            "# При выполнении работ принять меры по обеспечению целостности и сохранности существующих сооружений и коммуникаций, находящихся в зоне производства работ",
                            "# При производстве сварочных работ предусмотреть комплекс организационно-технических мероприятий по обеспечению пожарной безопасности",
                            "# На всех этапах ведения демонтажных и монтажных работ обеспечить устойчивость и неизменяемость положения существующих конструкций",
                            "# Перед началом работ необходимо выполнить очистку конструкций в зоне ремонта от мусора, грязи, технологической пыли",
                            "# Исполнитель несет ответственность за соответствие выполненных работ чертежам. Всякое отступление от чертежей при производстве работ должно быть согласовано с организацией-разработчиком данной рабочей документации",
                            "Характеристики конструкций",
                            "- коэффициент надежности по ответственности – {}",
                            "- степень агрессивного воздействия среды – {}",
                            "- расчетная температура эксплуатации – {}",
                            "Материал конструкций",
                            "Марки сталей элементов конструкций приняты в зависимости от группы конструкций с учетом расчетной температуры и приведены в таблицах \"Ведомость элементов\", на схемах расположения элементов конструкций, на чертежах элементов и узлов конструкций и в спецификации металлопроката",
                            "Указания по разработке КМД и ППР",
                            "Отступления от рабочих чертежей КМ не допускаются. В случае необходимости эти отступления должны быть согласованы с организацией-разработчиком рабочих чертежей марки \"КМ\"",
                            "Указания по изготовлению и монтажу металлоконструкций",
                            "# Изготовление металлоконструкций производить в соответствии с требованиями ГОСТ 23118-2012, СП 53-101-98 \"Изготовление и контроль качества стальных строительных конструкций\", указаниями в чертежах марки \"КМ\"",
                            "# Монтаж металлоконструкций производить в соответствии с требованиями СП 70.13330.2012 \"Несущие и ограждающие конструкции. Актуализированная редакция СНиП 3.03.01-87\", СНИП 12-04-2002 \"Безопасность труда в строительстве. Часть 2. Строительное производство\" и указаниями в чертежах марки \"КМ\"",
                            "# Отклонения металлоконструкций при изготовлении и монтаже не должны превышать предельных значений, указанных в СП 53-101-98, СП 70.13330.2012 и в чертежах марки \"КМ\" данного основного комплекта",
                            "Крепление элементов",
                            "Минимальное усилие для расчетного прикрепления элементов – 5 тс",
                            "Соединение элементов металлоконструкций",
                            "- сварные",
                            "- на болтах нормальной точности по ГОСТ Р ИСО 4014-2013",
                            "# Материалы для сварки принимать по таблице Г.1 СП 16.13330.2017 в зависимости от марок стали свариваемых элементов",
                            "# Монтажные швы выполнять ручной сваркой ГОСТ 9467-75 \"Электроды покрытые металлические для ручной дуговой сварки конструкционных и теплоустойчивых сталей. Типы\":\nДля сталей C245 сварку производить электродами типа 342A",
                            "# Применение прерывистых швов не допускается",
                            "# Контроль качества монтажных сварных соединений должен осуществляться в соответствии с требованиями п. 10.4 СП 70.13330.2012",
                            "# Болты класса точности B (нормальной точности) M20 приняты по ГОСТ Р ИСО 4014-2013 \"Болты с шестигранной головкой. Классы точности A и B\", класса прочности 8.8 по ГОСТ ISO 898-1-2014 \"Механические свойства крепежных изделий из углеродистых и легированных сталей. Часть 1. Болты, винты и шпильки установленных классов прочности с крупным и мелким шагом резьбы\"",
                            "# Гайки тип 1 класса точности B приняты по ГОСТ ISO 4032-2014. При работе болтов на срез и растяжение класс прочности гаек принят в соответствии с классом прочности болтов: 5 – при 5.6",
                            "Шайбы приняты круглые по ГОСТ 11371-78 исполнения 1 класса точности A",
                            "Для болтов класса точности B диаметр отверстий в конструкциях, работающих на срез и смятие, принят d=dв+2",
                            "В расчетных соединениях для болтов класса точности B разность номинальных диаметров отверстий и болтов не должна превышать 3 мм с учетом предельных отклонений диаметра отверстия и их овальности 0; +0,6; и 0; +1,0 по табл. 2 ГОСТ 23118-2012",
                            "Антикоррозионная защита",
                            "# Антикоррозионная защита запроектирована в соответствии с требованиями СП 28.13330.2017 \"Защита строительных конструкций от коррозии\"",
                            "# Антикоррозионную защиту производить в соответствии с требованиями СП 72.13330.2016 \"Защита строительных конструкций и сооружений от коррозии. СНиП 3.04.03-85\" и ГОСТ 12.3.005-75 \"Система стандартов безопасности труда (ССБТ). Работы окрасочные. Общие требования безопасности\"",
                            "# Подготовка поверхностей перед окраской должна осуществляться в соответствии с требованиями ГОСТ 9.402-2004 путем удаления ржавчины и прокатной окалины с помощью ручного или механизированного абразивного инструмента и очистки жировых и прочих загрязнений с помощью волосяных кистей или ветоши, смоченных в уайт-спирите. Перед нанесением антикоррозионного покрытия поверхности конструкций обеспылить",
                            "# Защита металлоконструкций от коррозии осуществляется окраской лакокрасочными материалами группы III: эмалью XB-7141 по ТУ 2313-016-75351875-2005 в 4 слоя по 2 слоям грунтовки XC-010 по ТУ 6-21-7-89",
                            "Общая толщина покрытия – 160 мкм",
                            "Степень очистки поверхности стальных конструкций от окислов перед окраской – 2 по ГОСТ 9.402-2004",
                            "Допускается замена лакокрасочных материалов на сертифицированные материалы с аналогичными техническими характеристиками",
                            "Цвет окраски антикоррозионного покрытия принимать в соответствии с цветом окраски существующих конструкций",
                            "Перечень видов работ, на которые должны быть составлены акты освидетельствования скрытых работ:",
                            "- подготовка поверхности металлоконструкций под окраску",
                            "- приемка грунтовочного слоя антикоррозионного покрытия",
                            "- визуальный контроль качества сварных швов"],
                   'order_num': [1, 2, 3, 4, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 1, 2, 3, 4, 5, 6, 7, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 1, 2, 1, 2, 1, 2, 3, 4, 1, 2, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4]})

In [566]:
df2 = df.copy()

In [567]:
df2['section_id'] += df_sections_len

In [568]:
df = df.append(df2, ignore_index=True)

In [569]:
df.insert(0, 'id', range(1, len(df) + 1))

In [570]:
cursor.execute(open("sql_server/35_2.sql", "r").read())
conn.commit()

In [571]:
execute_values(df, "general_data_points")

In [572]:
conn.close()